<a href="https://colab.research.google.com/github/NasrinGitHub/02-KaggleProjs/blob/main/Untitled0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

import seaborn as sns
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report

import warnings
warnings.filterwarnings('ignore')

import pickle
import gensim

In [2]:
from sklearn.model_selection import train_test_split

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.ensemble import RandomForestClassifier

In [5]:
# EDA and Preprocessing the data

In [3]:
df = pd.read_csv("IMDB_Dataset.csv",engine="python", on_bad_lines='skip' )
df

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive
...,...,...
49995,I thought this movie did a down right good job...,positive
49996,"Bad plot, bad dialogue, bad acting, idiotic di...",negative
49997,I am a Catholic taught in parochial elementary...,negative
49998,I'm going to have to disagree with the previou...,negative


In [4]:
dups= df[df.duplicated()].review
dups

3537     Quite what the producers of this appalling ada...
3769     My favourite police series of all time turns t...
4391     Beautiful film, pure Cassavetes style. Gena Ro...
6352     If you liked the Grinch movie... go watch that...
6479     I want very much to believe that the above quo...
                               ...                        
49912    This is an incredible piece of drama and power...
49950    This was a very brief episode that appeared in...
49984    Hello it is I Derrick Cannon and I welcome you...
49986    This movie is a disgrace to the Major League F...
49991    Les Visiteurs, the first movie about the medie...
Name: review, Length: 418, dtype: object

In [ ]:
#allDups = df[df["review"].isin(dups)]
#allDups.groupby("review")['sentiment'].apply(list)

In [5]:
data = df.drop_duplicates()
data.shape

(49582, 2)

In [6]:
data.isna().sum()

review       0
sentiment    0
dtype: int64

In [10]:
data.nunique()

review       49582
sentiment        2
dtype: int64

In [7]:
data['sentiment'].value_counts()

sentiment
positive    24884
negative    24698
Name: count, dtype: int64

In [8]:
import nltk
import re
from nltk.corpus import stopwords
nltk.download('stopwords')
from nltk.stem import WordNetLemmatizer #lemmatization purpose

lemmatizer=WordNetLemmatizer()

# Preprocess function to clean text
def preprocess_text(text):
    #text = re.sub(r'\W', ' ', text)
    text = re.sub("[^a-zA-Z]", " ", text)
    text = text.lower()
    text = text.split()
    text = [word for word in text if word not in stopwords.words('english')]
    text = ' '.join(text)
    return text

data['review'] = data['review'].apply(preprocess_text)
reviews = data['review'].values
sentiments = data['sentiment'].values

# Convert sentiments to binary values
labels = np.array([1 if sentiment == 'positive' else 0 for sentiment in sentiments])


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [9]:

# Split the data
train_reviews, test_reviews, train_labels, test_labels = train_test_split(reviews, labels, test_size=0.2, random_state=42)

# Using CountVectorizer and RandomForest

In [15]:
## Use BAG OF WORDS for Vectorization
countvector=CountVectorizer(ngram_range=(2,2))
traindataset=countvector.fit_transform(train_reviews)

In [16]:
# Train RandomForest Classifier
randomclassifier=RandomForestClassifier(n_estimators=200,criterion='entropy')
randomclassifier.fit(traindataset,train_labels)

RandomForestClassifier(criterion='entropy')

In [17]:
#test the mdoel
test_dataset = countvector.transform(test_reviews)
predictions = randomclassifier.predict(test_dataset)

In [19]:
## Check accuracy
matrix=confusion_matrix(test_labels,predictions)
print(matrix)
score=accuracy_score(test_labels,predictions)
print(score)
report=classification_report(test_labels,predictions)
print(report)

[[3653 1286]
 [ 610 4368]]
0.8088131491378441
              precision    recall  f1-score   support

           0       0.86      0.74      0.79      4939
           1       0.77      0.88      0.82      4978

    accuracy                           0.81      9917
   macro avg       0.81      0.81      0.81      9917
weighted avg       0.81      0.81      0.81      9917



# Using Tokenizer and Embedding layer

In [10]:

# Step 2: Preprocess the Data
import tensorflow as tf

from tensorflow.keras.layers import Embedding
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import SimpleRNN
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import GRU
from tensorflow.keras.layers import Bidirectional
from tensorflow.keras.layers import Flatten

from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import Tokenizer

from tensorflow.keras.initializers import Constant
from sklearn.naive_bayes import MultinomialNB
from sklearn import svm


In [11]:
# Parameters
vocab_size = 10000
max_length = 100
embedding_dim = 100
trunc_type = 'post'
oov_tok = "<OOV>"

In [44]:
# Tokenizer
tokenizer = Tokenizer(num_words=vocab_size, oov_token=oov_tok)
tokenizer.fit_on_texts(train_reviews)
word_index = tokenizer.word_index

# Tokenize and create pad sequences
train_sequences = tokenizer.texts_to_sequences(train_reviews)
train_padded = pad_sequences(train_sequences, maxlen=max_length, truncating=trunc_type)

test_sequences = tokenizer.texts_to_sequences(test_reviews)
test_padded = pad_sequences(test_sequences, maxlen=max_length, truncating=trunc_type)


In [45]:
# Convert labels to numpy arrays
train_labels = np.array(train_labels)
test_labels = np.array(test_labels)

# Model definition
model = Sequential([
    Embedding(vocab_size, embedding_dim, input_length=max_length),
    LSTM(64, return_sequences=True),
    LSTM(32),
    Dense(1, activation='sigmoid')
])

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

# Training the model
num_epochs = 5
history = model.fit(train_padded, train_labels, epochs=num_epochs, validation_data=(test_padded, test_labels))

Epoch 1/5
1240/1240 [==============================] - 215s 171ms/step - loss: 0.3349 - accuracy: 0.8561 - val_loss: 0.3241 - val_accuracy: 0.8756
Epoch 2/5
1240/1240 [==============================] - 138s 111ms/step - loss: 0.2191 - accuracy: 0.9165 - val_loss: 0.3111 - val_accuracy: 0.8769
Epoch 3/5
1240/1240 [==============================] - 121s 98ms/step - loss: 0.1595 - accuracy: 0.9414 - val_loss: 0.3230 - val_accuracy: 0.8691
Epoch 4/5
1240/1240 [==============================] - 116s 94ms/step - loss: 0.1164 - accuracy: 0.9589 - val_loss: 0.4192 - val_accuracy: 0.8687
Epoch 5/5
1240/1240 [==============================] - 115s 93ms/step - loss: 0.0896 - accuracy: 0.9691 - val_loss: 0.4130 - val_accuracy: 0.8743


# Using Word2Vec model and LSTM layer

In [12]:
from nltk.corpus import stopwords
from gensim.models import Word2Vec

In [13]:
# Tokenize the reviews for Word2Vec
train_tokens = [review.split() for review in train_reviews]

# Train Word2Vec model
word2vec_model = Word2Vec(sentences=train_tokens, vector_size=embedding_dim, window=5, min_count=1, workers=4)

# Create a word index dictionary
word_index = {word: index for index, word in enumerate(word2vec_model.wv.index_to_key)}

# Function to convert texts to sequences of word vectors
def text_to_sequence(text, word2vec_model, word_index, max_length):
    sequence = []
    for word in text.split():
        if word in word_index:
            sequence.append(word2vec_model.wv[word])
        else:
            sequence.append(np.zeros(word2vec_model.vector_size))
    if len(sequence) < max_length:
        sequence += [np.zeros(word2vec_model.vector_size)] * (max_length - len(sequence))
    else:
        sequence = sequence[:max_length]
    return np.array(sequence)

# Convert all reviews to sequences of word vectors
#max_length = 100
train_sequences = np.array([text_to_sequence(review, word2vec_model, word_index, max_length) for review in train_reviews])
test_sequences = np.array([text_to_sequence(review, word2vec_model, word_index, max_length) for review in test_reviews])

In [14]:
# Define the LSTM model
model = Sequential([
    LSTM(64, input_shape=(max_length, word2vec_model.vector_size), return_sequences=True),
    LSTM(32),
    Dropout(0.5),
    Dense(1, activation='sigmoid')
])

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(train_sequences, train_labels, epochs=10, batch_size=64, validation_data=(test_sequences, test_labels))

# Evaluate the model
test_loss, test_acc = model.evaluate(test_sequences, test_labels)
print(f'Test Accuracy: {test_acc}')

Epoch 1/10
620/620 [==============================] - 15s 15ms/step - loss: 0.4449 - accuracy: 0.8005 - val_loss: 0.3545 - val_accuracy: 0.8488
Epoch 2/10
620/620 [==============================] - 8s 12ms/step - loss: 0.3433 - accuracy: 0.8554 - val_loss: 0.3351 - val_accuracy: 0.8590
Epoch 3/10
620/620 [==============================] - 7s 12ms/step - loss: 0.3218 - accuracy: 0.8654 - val_loss: 0.3221 - val_accuracy: 0.8623
Epoch 4/10
620/620 [==============================] - 8s 12ms/step - loss: 0.3029 - accuracy: 0.8760 - val_loss: 0.3143 - val_accuracy: 0.8672
Epoch 5/10
620/620 [==============================] - 8s 12ms/step - loss: 0.2879 - accuracy: 0.8833 - val_loss: 0.3198 - val_accuracy: 0.8652
Epoch 6/10
620/620 [==============================] - 7s 12ms/step - loss: 0.2751 - accuracy: 0.8882 - val_loss: 0.3224 - val_accuracy: 0.8677
Epoch 7/10
620/620 [==============================] - 8s 12ms/step - loss: 0.2588 - accuracy: 0.8967 - val_loss: 0.3120 - val_accuracy: 0.865

# Using Fine tunned Word2Vec, Embedding and  Bi-LSTM

In [19]:
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [20]:
# Test & Train size
TEST_SIZE = 0.3
EPOCHS = 5
BATCH_SIZE = 100

In [21]:
corpus = reviews

In [22]:
def generate_word2vec_tokens(list_of_sentences):
  sentences = []
  for s in list_of_sentences:
    sentences.append(s.split())
  return sentences

def generate_word2vec(sentences):
  w2v_model=gensim.models.Word2Vec(sentences=sentences,vector_size=embedding_dim,window=10,min_count=1)
  w2v_model.train(sentences,epochs=10,total_examples=len(sentences))
  return w2v_model

def save_word2vec(word2vec_model, model_name):
  word2vec_model.wv.save_word2vec_format(model_name, binary=True)

def load_word2vec(model_name):
  loaded_model = gensim.models.KeyedVectors.load_word2vec_format(model_name, binary=True)
  return loaded_model

In [23]:
ss = generate_word2vec_tokens(corpus)
word2vec_model  = generate_word2vec(ss)


In [24]:
save_word2vec(word2vec_model , 'word2vec.bin')

w2v_model = load_word2vec('word2vec.bin')

In [25]:
def apply_tokenizer(data):
  tok = Tokenizer(num_words = vocab_size)
  tok.fit_on_texts(data)
  encoded_data = tok.texts_to_sequences(data)
  return [tok, encoded_data]

def apply_padding(data):
  pad_data = pad_sequences(data, maxlen=max_length)
  print("New Dimensions: ", pad_data.shape)
  return pad_data

def generate_embedded_matrix(tok, w2v_model):
  embed_matrix=np.zeros(shape=(vocab_size,embedding_dim))
  for word,i in tok.word_index.items():
    if i < vocab_size and  word in w2v_model.key_to_index:
      embed_vector=w2v_model.get_vector(word)
      if embed_vector is not None:  # word is in the vocabulary learned by the w2v model
        embed_matrix[i]=embed_vector
  return embed_matrix

In [26]:
tok, encoded_corpus = apply_tokenizer(corpus)

embed_matrix = generate_embedded_matrix(tok, w2v_model)

pad_corpus = apply_padding(encoded_corpus)



New Dimensions:  (49582, 500)


In [42]:
X_final=pad_corpus
y_final= np.array(sentiments)

In [33]:
y = np.array([1 if y == 'positive' else 0 for y in y_final])
y_final = np.asarray(y).astype("float64")

In [39]:
y_final

array([1., 1., 1., ..., 0., 0., 0.])

In [35]:
X_final.shape,y_final.shape

((49582, 500), (49582,))

In [36]:
X_train, X_test, y_train, y_test = train_test_split(X_final, y_final, test_size=TEST_SIZE, random_state=42)

In [40]:
# Create Model
def create_model(MODEL):
  model=Sequential()
  model.add(Embedding(input_dim=vocab_size,output_dim=embedding_dim,input_length=max_length,embeddings_initializer=Constant(embed_matrix)))
  model.add(Dense(256))
  model.add(Dropout(0.05))
  model.add(MODEL)
  model.add(Dense(1,activation='sigmoid'))
  model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
  model.summary()
  return model

# Train Model
def train_model(MODEL, x_train, y_train):
  earlystop = tf.keras.callbacks.EarlyStopping(monitor='val_loss', min_delta=0, patience=3, verbose=0, mode='auto')
  MODEL.fit(
      x_train,
      y_train,
      validation_split=0.1,
      epochs=EPOCHS,
      shuffle=True,
      batch_size=BATCH_SIZE
      ,callbacks=[earlystop]
  )

def generate_classification_report(y_test, y_pred):
  print(classification_report(y_test,y_pred))

# Test Model
def test_model(MODEL, model_name, x_test, y_test):
  y_pred=MODEL.predict(x_test)
  y_pred=np.where(y_pred >= 0.5, 1,0)
  score = accuracy_score(y_pred,y_test)
  print('Model Accuracy: ',score)
  print("\n\nClassification Report")
  generate_classification_report(y_test,y_pred)


In [41]:
bi_lstm = create_model(Bidirectional(LSTM(256)))
train_model(bi_lstm, X_final, y_final)
test_model(bi_lstm,"Bidirectional_LSTM_RNN", X_test, y_test)

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 500, 100)          1000000   
                                                                 
 dense_1 (Dense)             (None, 500, 256)          25856     
                                                                 
 dropout_1 (Dropout)         (None, 500, 256)          0         
                                                                 
 bidirectional (Bidirection  (None, 512)               1050624   
 al)                                                             
                                                                 
 dense_2 (Dense)             (None, 1)                 513       
                                                                 
Total params: 2076993 (7.92 MB)
Trainable params: 2076993 (7.92 MB)
Non-trainable params: 0 (0.00 Byte)
________________